<a href="https://colab.research.google.com/github/Mostofa-Najmus-Sakib/Search-Engine-from-scratch/blob/main/Codes/search_engine_finalwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.simplefilter('ignore')
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
cd /content/drive/Shared\ drives/2021_IR_437_537/Submission/Project_1/

In [ ]:
import pandas as pd
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from collections import Counter
import collections
import numpy as np

import pandas as pd
import numpy as np
import pandas as pd
import nltk
import re
import string
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
string.punctuation
stopwords = nltk.corpus.stopwords.words('english')
pd.set_option('display.max_rows', 100)
import pickle
import heapq
from collections import Counter
import collections
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

from functools import reduce

In [ ]:
# df_original = pd.read_csv( 'project_1_Wiki_sample.csv')
Q= pd.read_csv('querymergedata.csv')
Q = Q[[ 'qid', 'AnonID', 'Query', 'QueryTime']]

In [ ]:
df = pd.read_pickle('preprocessed_2_4_6_merge_pickle.pkl')



In [ ]:
df

,content,title,id,content_new
0,Morocco–Saudi Arabia relations\r\n\r\nMoroccan...,Morocco–Saudi Arabia relations,1,"[moroccan, saudi, arabian, relat, refer, curre..."
1,Anthony United States Post Office\r\n\r\nThe A...,Anthony United States Post Office,2,"[anthoni, unit, state, post, offic, list, nati..."
2,Dumraon (Vidhan Sabha constituency)\r\n\r\nDum...,Dumraon Vidhan Sabha constituency,3,"[dumraon, vidhan, sabha, constitu, one, legisl..."
3,Chaker Khazaal\r\n\r\nChaker Khazaal (born 28 ...,Chaker Khazaal,4,"[chaker, khazaal, born, septemb, beirut, leban..."
4,Vicente Pascual Pastor\r\n\r\nVicente Pascual ...,Vicente Pascual Pastor,5,"[vicent, pascual, pastor, alcoy, june, alcoy, ..."
...,...,...,...,...
599995,W50\r\n\r\nW50 may refer to:,W50,599996,"[w, may, refer]"
599996,Medical logic module\r\n\r\nA medical logic mo...,Medical logic module,599997,"[medic, logic, modul, mlm, independ, unit, hea..."
599997,Paderborn Untouchables\r\n\r\nThe Untouchables...,Paderborn Untouchables,599998,"[untouch, paderborn, full, name, untouch, pade..."
599998,"Sussex Corner, New Brunswick\r\n\r\nSussex Cor...",Sussex Corner New Brunswick,599999,"[sussex, corner, popul, canadian, villag, king..."


In [ ]:
df.shape

(600000, 4)

In [ ]:
df_merge_5 = pd.read_pickle('inverted_0_600000_merge.pkl')

In [ ]:
len(df_merge_5)

1134085

## Inverted index

In [ ]:
# df_merge_1 = pd.read_csv('inverted_0_10000.csv')
# df_merge_2 = pd.read_csv('inverted_10000_20000.csv')

In [ ]:
# def string_to_list_of_inverted_index(string_of_list):
#   item_1 = string_of_list[1:-1].replace("'", "").split(", ")
#   item_2 = [int(x) for x in item_1]
#   return item_2

In [ ]:
# df_merge_5 = pd.concat([df_merge_1, df_merge_2]).reset_index()

In [ ]:
# df_merge_5['index'] = df_merge_5['id'].apply(string_to_list_of_inverted_index)
# df_merge_5

In [ ]:
# df_merge_5 = df_merge_5.groupby([df_merge_5.word]).index.apply(sum).to_frame().reset_index()
# df_merge_5

## Text Preprocessing (Functions)

In [ ]:
#### First 2 function is added from the snippet generation portion


def remove_title(title, content):
  character_remove = len(title)
  content = content[character_remove:]
  return content

def split_it(a):
    return  re.sub('[\r\n\r\n]', ' ', a)


def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

def preprocess(sentence):# remove number
    sentence=str(sentence)
    rem_num = re.sub('[0-9]+', '', sentence)
    return rem_num

def preprocess1(sentence):
    sentence= str(sentence)
    rem_num = re.sub('[^a-zA-Z0-9]+',' ', sentence)
    cleaned_string = re.sub('\s+',' ', rem_num)
    return cleaned_string 

def remove_stopwords(text):#stopword remove
    output= [i for i in text if i not in stopwords]
    return output

#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

porter_stemmer = PorterStemmer()
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

## Query Suggestion (Functions)

In [ ]:
# Q= pd.read_csv('querymergedata.csv')
# Q = Q[[ 'qid', 'AnonID', 'Query', 'QueryTime']]

In [ ]:
def qs(ql):
    
    r, c = Q.shape
    a = []

    for i in range(r):
        s1= Q.iloc[i,2]
        jy= (s1.split())
        if jy[0]==ql:                                               ############## for now we can focus in this
            a.append(Q.iloc[i,0])
    vq = pd.DataFrame(columns =['qid', 'AnonID', 'Query', 'QueryTime'])
    
    for i in a:
        vq = vq.append(Q[Q['qid'] == i])
    vq = vq[vq.groupby('AnonID').AnonID.transform('count') > 1] 
    ### removing all the sessions that appeared once (check with the prof)
    r1,c1 = vq.shape
    a1 = []
    b1 = []#qid
    
    for i in range(r1):
        s1= vq.iloc[i,2]
        jy= (s1.split())
    # print(len(jy))
        l= len(jy)
        if l==1:                                               ############## might need to change here, check again with the length
        # print(vq.iloc[:,2])
            a1.append(vq.iloc[i,0])
            
    vq_val = pd.DataFrame(columns =['qid', 'AnonID', 'Query', 'QueryTime'])
    for i in a1:
        vq_val = vq_val.append(vq[vq['qid'] == i])
    p = vq_val['AnonID'].unique()
    vq_val_2 = pd.DataFrame(columns =['qid', 'AnonID', 'Query', 'QueryTime'])
    
    for i in p:
        vq_val_2 = vq_val_2.append(vq[vq['AnonID'] == i])
        
    unique_query = vq_val_2['Query'].value_counts()
    
    df_Score  = unique_query.to_frame()
    
    freq_first_query_word = unique_query.iloc[0]
    
    df_Score['Score'] =   df_Score['Query']  / freq_first_query_word
    
    u = df_Score.reset_index().head()
    
    
    sug = u[['index','Score']]
    
    sug.rename(columns={'index':'Query suggestion'}, inplace=True)
    
    print(sug)
    
    wait = input('....Press any key to continue....')
   # return(sug)

In [ ]:
def query_suggestion(y):
    
    qu = nltk.word_tokenize(y)
    
    
    print('Query suggestion')
    
        
    l= qu[0].lower() 
    
    qqq = qs(l)
    
    
    
    return qqq

## Identifying candidate resource and relevance ranking (Functions)

#### Maxd

In [ ]:
def maxis(names):
    nameset=set(names)
   # print(nameset)
    d={name:names.count(name) for name in nameset}

    d1 = {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}
    # print(d1)
    if (len(d1))==0:
      d1.clear()
    #  print('empty')

    else:

      d2 =list(d1.values())[-1]
      return d2

In [ ]:
def query_preprocess(x):
    df1 = pd.DataFrame({'real_query':x.splitlines()})
    df1['real_query']= df1['real_query'].apply(lambda x:remove_punctuation(x))
    df1['real_query'] = df1['real_query'].apply(lambda x: x.lower())#lower
    df1['real_query'] = df1['real_query'].map(lambda s:preprocess(s))
    df1['real_query'] = df1['real_query'].map(lambda s:preprocess1(s))
    df1['real_query'] = df1.apply(lambda row: nltk.word_tokenize(row['real_query']), axis=1)#tokenize1
    df1['real_query'] = df1['real_query'].apply(lambda x:remove_stopwords(x))
    df1['real_query'] = df1['real_query'].apply(lambda x:lemmatizer(x))
    df1['real_query'] = df1['real_query'].apply(lambda x: stemming(x))
    for i in range(1):
        qq = df1.iloc[0,0]
    return qq

In [ ]:
def dr(x, qq, df, df_merge_3):
    # merge query and inverted index
    q_df= pd.DataFrame(qq, columns=['word'])
    q_df_merge = q_df.merge(df_merge_3, on ='word')
    d900 = pd.DataFrame(columns = ['content', 'title', 'id', 'content_new'])
    total = len(df)
    if (len( q_df_merge))==1:
      m =   q_df_merge.iloc[0,1]
      idf_bottom= (len(q_df_merge.iloc[0,1]))
      idf = np.log2(total /  idf_bottom)
    else:
    
      # print(q_df_merge)
      m1 =[]# allid
    #  d900 = pd.DataFrame(columns = ['content', 'title', 'id', 'content_new'])
      
      idf_bottom = []
      for i in range (len(q_df_merge)):
          m1.append(q_df_merge.iloc[i,1])
          idf_bottom.append(len(q_df_merge.iloc[i,1]))

      # for y in m1:
      #   print('all result',y)
     # print(m1)
  #    print(len(m1))
      
    #  print('idf is',idf_bottom)
     # total = len(df)
    #  print('total',total)
      idf = []
      # print('idf bottom 2 values ',idf_bottom)
      for no in idf_bottom:
          g = np.log2(total / no)
          idf.append(g)

   #print('idf value:', idf)
      m = list(reduce(set.intersection, [set(item) for item in m1 ]))
  #find the doc id where the query terms appear
   #   print('commom id is',m)
   #   print(len(m))
    # for i5 in m:
    #         print('common collection matched is',i5)

    print('idf value :',idf)
    for i9 in m:
        d900= d900.append(df[df['id']==i9])# get all doc where all query term appears
    if (len(d900))==0:
        print('No Resource Found')
    else:
        d231= d900.copy()
        #print(d231)
        d231.insert(0, 'new_id', range(1, 1+ len(d231)))
        # print(len(d231))
        d231.dropna(inplace= True)
        # print(len(d231))
        # try: 
        d231['maxd']= d231['content_new'].apply(maxis)
        # print(len(d231))
        d231.dropna(inplace= True)
       

        f = d231.reset_index(drop=True)
        # print(f.iloc[108:111,3])
        # print('f whole result', f)

        
        tf= []
        for i in range(len(f)):
       #   try:
                names = f.iloc[i,4]
                nameset=set(names)
                d={name:names.count(name) for name in nameset}
                #print(i)
                d1 = {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}
                # print(d1)
                c = 0
                ii = 0
                if (len(qq))!=1:

                  for s in qq:
                    if s in d1.keys():
                  
                  
                        # print('len not 1 i is',s)
                        value = d1[s]
                        # print('value is :', value )
                        # print('idf is :', idf )
                      # print( s, 'frequency is',value)
                        div = value*idf[ii]
                       # print('div is',div)
                        ii = ii + 1
                        c= c+ div
                       # print('sum is',c)
                  tf.append(c)
                else:
                  for s in qq:
                    if s in d1.keys():
                  
                  
                        # print('inly one key',s)
                        value = d1[s]
                       # print('value is :', value )
                        # print('idf is :', idf )
                      # print( s, 'frequency is',value)
                        div = value*idf
                        # print(div)
                        #ii = ii + 1
                        # c= c+ div
                      # print(c)
                  tf.append(div)
                #print('sum is', tf)

         
        o= len(tf)
        tfcol = pd.DataFrame(np.array(tf).reshape(o,1), columns = list("a"))
        jk = pd.concat([f, tfcol], axis=1)
      #  print(jk)
        jk['score'] = jk['a']/jk['maxd']

     #   print('raw dataframe',jk[['maxd','a','score']]) need to show professor

###new
        #idf = np.log(len(df)/len(jk))
    #idf calculation
        #jk['score'] =  jk['score'] *idf # multiply to calculate tfidf
#######
        jk = jk.sort_values(["score"], ascending=False)
        jki = jk.reset_index(drop=True)

        jkicol = jki[['maxd','a','score']]
       # print

        jki= jki[['id','content','title','score']]
        totalrs = len(jki)
        print("Total resource found : " ,totalrs)
        jk_is= jki.head()
        jk_is.dropna(subset = ["content"], inplace=True)
        # jk_is.to_csv('cosim.csv')
        kkk = jk_is[['id','content']]
        print('-----Top Relevant Documents-----')
        print(kkk)
        print('\n')
        hhhh = jk_is[['title','score']]
        print(hhhh)
        print('\n')


        query_frequency,que_tf, query1 = query_preprocess_for_cosine_similarity(x)
#  print(query1)
     
        cosin= cosim(jk_is, query_frequency,que_tf, query1 )

# print(cosin)
        for cc in range(len(jk_is)):
            print('\n')
            print('docid is')
            print(cosin.iloc[cc,0])
            print('\n')
            print('cosime similarity sentence')
            print(cosin.iloc[cc,1])
        wait = input('....Press any key to continue....')


## Generating sinppets (Functions)

In [ ]:
def query_preprocess_for_cosine_similarity(query_as_string):
  query_lower = remove_punctuation(query_as_string).lower()
  query_remove_punc = preprocess1(query_lower)
  query_tok = nltk.word_tokenize(query_remove_punc)
  query_tok_stop_word_remove = remove_stopwords(query_tok)
  query_tok_lemm = lemmatizer(query_tok_stop_word_remove)
  query_tok_stemm = stemming(query_tok_lemm)
  query1 = query_tok_stemm

  query_frequency = collections.Counter(query1)
  que_tf = {k: v / sum(query_frequency.values()) for k, v in query_frequency.items()}

  return query_frequency, que_tf, query1



def remove_title(title, content):
  character_remove = len(title)
  content = content[character_remove:]
  return content

def split_it(a):
    return  re.sub('[\r\n\r\n]', ' ', a)

def cal_quer_doc_freq(list_of_lists, query1):
  list_containing_query_term = []
  for list1 in list_of_lists:
    list1_1 = remove_punctuation(list1).lower()
    list1_2 = preprocess1(list1_1)
    list_tok_1 =  nltk.word_tokenize(list1_2) 
    list_tok_stop_word_remove = remove_stopwords(list_tok_1)
    list_tok_lemm = lemmatizer(list_tok_stop_word_remove)
    list_tok_stemm = stemming(list_tok_lemm)
    list_tok = list_tok_stemm

    # list_tok =  nltk.word_tokenize(list1) ## only this line runs if any of the preprocess is not added 

    if any(item in list_tok for item in query1) == True:
      query_in_sen = [x for x in list_tok if x in query1]
      res = {idx : query_in_sen.count(idx) for idx in set(query1)}
      res['Sentence'] = list1
      res_copy = res.copy()
      list_containing_query_term.append(res_copy)
    
  return list_containing_query_term


def cal_term_freq_in_doc_across_sen(list_of_lists, query1):
  list_containing_query_term_2 = []

  for list1 in list_of_lists:
    list1_1 = remove_punctuation(list1).lower()
    list1_2 = preprocess1(list1_1)
    list_tok_1 =  nltk.word_tokenize(list1_2) 
    list_tok_stop_word_remove = remove_stopwords(list_tok_1)
    list_tok_lemm = lemmatizer(list_tok_stop_word_remove)
    list_tok_stemm = stemming(list_tok_lemm)
    list_tok = list_tok_stemm

    # list_tok =  nltk.word_tokenize(list1) ## only this line runs if any of the preprocess is not added 

    if any(item in list_tok for item in query1) == True:   
      query_in_sen = [x for x in list_tok if x in query1]
      res = {idx : query_in_sen.count(idx) for idx in set(query1)}
      res['Sentence'] = list1
      res_copy = res.copy()
      list_containing_query_term_2.append(res_copy)
    

    list_containing_query_term_on_all_sentence = []
    newDict = {}


    for item in list_containing_query_term_2:
      item.pop("Sentence", None)
      for (key, value) in item.items():
        if value >1:
          newDict[key] = 1
        else:
          newDict[key] = value
        newDict_copy = newDict.copy()
      list_containing_query_term_on_all_sentence.append(newDict_copy)

  counter = collections.Counter()
  for d in list_containing_query_term_on_all_sentence: 
    counter.update(d)  
  
  return counter


def que_doc_sum_across_doc(list_of_lists, query1):

  list_containing_query_term_3 = []

  for list1 in list_of_lists:

    list1_1 = remove_punctuation(list1).lower()
    list1_2 = preprocess1(list1_1)
    list_tok_1 =  nltk.word_tokenize(list1_2) 
    list_tok_stop_word_remove = remove_stopwords(list_tok_1)
    list_tok_lemm = lemmatizer(list_tok_stop_word_remove)
    list_tok_stemm = stemming(list_tok_lemm)
    list_tok = list_tok_stemm

    # list_tok =  nltk.word_tokenize(list1) ## only this line runs if any of the preprocess is not added

    if any(item in list_tok for item in query1) == True:   
      query_in_sen = [x for x in list_tok if x in query1]
      res = {idx : query_in_sen.count(idx) for idx in set(query1)}
      res_copy = res.copy()
      list_containing_query_term_3.append(res_copy)

  counter = collections.Counter()
  for d in list_containing_query_term_3: 
    counter.update(d)  
  
  return counter



def doc_que_idf(counter_collec_dic,number_of_sentence_to_count):

  number_of_sentence_in_collection = len(number_of_sentence_to_count)
  d = dict(counter_collec_dic)
  idf_value = {k: np.log( number_of_sentence_in_collection/(v+1) +1) for k, v in d.items()}
  return idf_value


def doc_tf(list_of_dic):

  tf_list = []
  for item in list_of_dic:

    list1 = item['Sentence']
    list1_1 = remove_punctuation(list1).lower()
    list1_2 = preprocess1(list1_1)
    list_tok_1 =  nltk.word_tokenize(list1_2) 
    list_tok_stop_word_remove = remove_stopwords(list_tok_1)
    list_tok_lemm = lemmatizer(list_tok_stop_word_remove)
    list_tok_stemm = stemming(list_tok_lemm)
    list_tok = list_tok_stemm
    length_of_sen = len(list_tok)


    item.pop("Sentence", None)
    tf_value = {k: v / length_of_sen for k, v in item.items()}  
    tf_list.append(tf_value)

  return tf_list



def doc_tf_idf(d_tf_list_of_dic, d_idf):

  d_tf_idf = []

  for item in d_tf_list_of_dic:
    tf_idf = {k: v * d_idf[k] for k, v in item.items()} 
    d_tf_idf.append(tf_idf)
  return d_tf_idf


def query_tf_idf(q_tf_dic, que_idf):
  q_tf_idf = {k: q_tf_dic[k]*que_idf[k] for k in que_idf}
  return q_tf_idf



def cosine_similarity(d_tf_idf, q_tf_idf):
  final_cosine_similarity = []
  for item in d_tf_idf:
    cos_sim_numerator = [v * q_tf_idf[k] for k, v in item.items()]
    cos_sim_numerator_without_nan = np.nansum(cos_sim_numerator)

    cosine_sim_denomenator_d =  [v * v for k, v in item.items()]
    cosine_sim_denomenator_q =  [q_tf_idf[k] * q_tf_idf[k] for k, v in item.items()]
    cosine_sim_denomenator_dq = np.sqrt(np.nansum(cosine_sim_denomenator_d)) * np.sqrt(np.nansum(cosine_sim_denomenator_q))

    cosine_similarity = cos_sim_numerator_without_nan/cosine_sim_denomenator_dq
    final_cosine_similarity.append(cosine_similarity)
    
  return(final_cosine_similarity)


def sentence_extraction(cos_similar_value, list_of_sen):
  cos_similarity_max_value = heapq.nlargest(2, cos_similar_value)
  index_list = []
    
  for item in cos_similarity_max_value:  
    index = cos_similarity_max_value.index(item)
    index_list.append(index)
  sentence_list =[]
  for ind in index_list:
    sentence = list_of_sen[ind]['Sentence']

    sentence_list.append(sentence)
  #sentence_with_cosine_value = []

  sentence_with_cosine_value = [cos_similarity_max_value, sentence_list]  ## removed cos_similarity_max_value as value was coming same, need to check the calculation
  return(sentence_with_cosine_value)


## Function to do all the preprocessing

In [ ]:
def text_preprocessing_all_function(df):

  df["content_new"] = df.apply(lambda x: remove_title(x["title"], x["content"]), axis = 1)
  df['content_new'] = df['content_new'].apply(split_it)

  df['content_new']= df['content_new'].apply(lambda x:remove_punctuation(x))
  df['content_new'] = df['content_new'].apply(lambda x: x.lower())#lower
  df['content_new'] = df['content_new'].map(lambda s:preprocess(s))
  df['content_new'] = df['content_new'].map(lambda s:preprocess1(s))
  df['content_new'] = df.apply(lambda row: nltk.word_tokenize(row['content_new']), axis=1)#tokenize
  df['content_new'] = df['content_new'].apply(lambda x:remove_stopwords(x))
  df['content_new'] = df['content_new'].apply(lambda x:lemmatizer(x))
  df['content_new'] = df['content_new'].apply(lambda x: stemming(x))
  return df

## Function to run all the cosine similarity calculation functions

In [ ]:
def cosim(df_doc, query_frequency,que_tf, query1 ):
   # query_frequency,que_tf, query1 = query_preprocess_for_cosine_similarity(original_query)
    df_doc["content_w_o_title"] = df_doc.apply(lambda x: remove_title(x["title"], x["content"]), axis = 1)
    df_doc['content_clean'] = df_doc['content_w_o_title'].apply(split_it)
    df_doc['con_clean_sen'] = df_doc['content_clean'].apply(sent_tokenize)
#     df_doc["que_sen_freq"] = df_doc["con_clean_sen"].apply(cal_quer_doc_freq)
    
    df_doc["que_sen_freq"] = df_doc.apply(lambda x: cal_quer_doc_freq(x['con_clean_sen'],query1), axis = 1)
    
#     df_doc["term_freq_in_sen"] = df_doc["con_clean_sen"].apply(cal_term_freq_in_doc_across_sen)
    
    df_doc["term_freq_in_sen"] = df_doc.apply(lambda x: cal_term_freq_in_doc_across_sen(x["con_clean_sen"],query1),axis=1)
    
    
#     df_doc["term_freq_in_sen_doc"] = df_doc["con_clean_sen"].apply(que_doc_sum_across_doc)
    
    df_doc["term_freq_in_sen_doc"] = df_doc.apply(lambda x: que_doc_sum_across_doc(x["con_clean_sen"],query1),axis = 1)
    
    
    
    # df_doc["q_d_idf"] = df_doc["term_freq_in_sen"].apply(doc_que_idf)
    df_doc["q_d_idf"] = df_doc.apply(lambda x: doc_que_idf(x["term_freq_in_sen"], x["que_sen_freq"]), axis =1)
    # df_doc["d_tf"] = df_doc.apply(lambda x: doc_tf(x["que_sen_freq"], x["term_freq_in_sen_doc"]), axis = 1)
    df_doc["d_tf"] = df_doc["que_sen_freq"].apply(doc_tf)
    df_doc["d_tf_idf"] = df_doc.apply(lambda x: doc_tf_idf(x["d_tf"], x["q_d_idf"]), axis = 1)
    df_doc["q_tf_idf"] = df_doc.apply(lambda x: query_tf_idf(que_tf, x["q_d_idf"]), axis = 1)
    df_doc["cos_similarity"] = df_doc.apply(lambda x: cosine_similarity(x["d_tf_idf"], x["q_tf_idf"]), axis = 1)
    
#     df_doc["que_sen_freq_for_sen"] = df_doc["con_clean_sen"].apply(cal_quer_doc_freq)
    
    df_doc["que_sen_freq_for_sen"] = df_doc.apply(lambda x: cal_quer_doc_freq(x["con_clean_sen"], query1),axis = 1)
    
    
    df_doc["cos_similarity_with_sentence"] = df_doc.apply(lambda x: sentence_extraction(x["cos_similarity"], x["que_sen_freq_for_sen"]), axis = 1)
    # df_doc= df_doc[['id','title', 'content', 'score', 'cos_similarity_with_sentence']] 
    df_doc= df_doc[['id', 'cos_similarity_with_sentence']] 
    #print('cos work',df_doc)
    return df_doc

In [ ]:
# pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 7)

## Call functions for results

In [ ]:
def main_menu():
    while True:
    
      print('\33[34m' "-----     Search Engine     -----")
      
      print('\33[34m' "\n1.  Query suggestion")
      
      print('\33[34m'  "\n2. Relevant document")
    
      choice = int(input('Enter your choice ...: '))
      
      if choice ==1:
            
            m= (input("Enter query for query suggestion:"))
            
            query_suggestion(m)

      if choice ==2:
            
            m1 =(input("Enter query to see relevance documents:"))
            l2 = query_preprocess(m1)
       
            dr(m1,l2,df, df_merge_5)
                                                                     
            
      
      if choice == 0:
          print('end of work')
          break

if __name__ == "__main__":
    main_menu()




-----     Search Engine     -----

1.  Query suggestion

2. Relevant document
idf value : 5.547706805634007
Total resource found :  12825
-----Top Relevant Documents-----
       id                                            content
0  514576  Jochen Neerpasch\r\n\r\nJochen Neerpasch (born...
1  297095  Courier (automobile)\r\n\r\nThe Courier was a ...
2  291968  Ferrari TR\r\n\r\nThe Ferrari TR, or 250 Testa...
3  492154  Welly (toy company)\r\n\r\nWelly Die Casting F...
4  292818  Auto Avio Costruzioni 815\r\n\r\nThe Auto Avio...


                       title     score
0           Jochen Neerpasch  5.547707
1       Courier  automobile   5.547707
2                 Ferrari TR  5.547707
3        Welly  toy company   5.547707
4  Auto Avio Costruzioni 815  5.547707




docid is
514576


cosime similarity sentence
[[1.0, 1.0], ['His racing career began in the 1960s, first on Borgward touring car, then with the 1964 24 Hours of Le Mans as a first major event.', 'His racing career began in t